# NYC Covid-19 Dashboard

## NYC Covid Data

For this task, we are going to scrape publically available covid-19 data in NYC.

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns # Using seaborn for visualization
import plotly.express as px
plt.rcParams['figure.figsize'] = [18, 6]

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

### COVID-19 Outcomes by Testing Cohorts: Cases, Hospitalizations, and Deaths
This will be the dataset to develop the overall NYC timeseries for Covid positivity

#### Import Data

In [3]:
chd = pd.read_csv('https://data.cityofnewyork.us/resource/cwmx-mvra.csv')

In [4]:
chd.head()

,extract_date,specimen_date,number_tested,number_confirmed,number_hospitalized,number_deaths
0,2020-08-28T00:00:00.000,2020-08-13T00:00:00.000,15344,208,17,1
1,2020-08-28T00:00:00.000,2020-04-09T00:00:00.000,9249,5064,1502,584
2,2020-08-28T00:00:00.000,2020-05-14T00:00:00.000,14232,1191,110,34
3,2020-08-28T00:00:00.000,2020-07-09T00:00:00.000,18181,351,34,5
4,2020-08-28T00:00:00.000,2020-08-06T00:00:00.000,15360,228,20,2


##### Data Structure and Schema

| Column Name            | Description                                                       | Data Type      |
| :-----------           | :-----------                                                      | :-----------   |
| extract_date           | Date of extraction from live disease surveillance database        | Date & Time    |
| specimen_date          | Date of specimen collection, equivalent to diagnosis date         | Date & Time    |
| number_tested          | Count of NYC residents newly tested for SARS-CoV-2                | Number         |
| number_confirmed       | Count of patients tested who were confirmed to be COVID-19 cases  | Number         |
| number_hospitalized    | Count of confirmed COVID-19 cases among patients ever hospitalized| Number         |
| number_deaths          | Count of confirmed COVID-19 cases among patients who died         | Number         |

#### Data Cleanup

Convert specimen_date to dt

In [5]:
chd['specimen_date_dt'] = [pd.to_datetime(row).strftime('%Y-%m-%d') for row in chd['specimen_date']]
chd['dt'] = [pd.to_datetime(row).strftime('%Y-%m-%d') for row in chd['extract_date']]

Select latest extract date

In [6]:
chd.head()

,extract_date,specimen_date,number_tested,number_confirmed,number_hospitalized,number_deaths,specimen_date_dt,dt
0,2020-08-28T00:00:00.000,2020-08-13T00:00:00.000,15344,208,17,1,2020-08-13,2020-08-28
1,2020-08-28T00:00:00.000,2020-04-09T00:00:00.000,9249,5064,1502,584,2020-04-09,2020-08-28
2,2020-08-28T00:00:00.000,2020-05-14T00:00:00.000,14232,1191,110,34,2020-05-14,2020-08-28
3,2020-08-28T00:00:00.000,2020-07-09T00:00:00.000,18181,351,34,5,2020-07-09,2020-08-28
4,2020-08-28T00:00:00.000,2020-08-06T00:00:00.000,15360,228,20,2,2020-08-06,2020-08-28


##### Add NYC data label to a new column called Boro

In [7]:
chd['boro'] = 'NYC'

In [8]:
# Get max dt of data load
chd = chd[(chd['dt'] == chd['dt'].max())]

##### Sort dataframe by specimen_date

In [9]:
chd.sort_values(by=['specimen_date'], inplace=True)
chd.reset_index(inplace=True, drop=True)

##### Add % Tested Positive

In [10]:
chd['pct_tested_positive'] = chd['number_confirmed']/chd['number_tested']

##### Add Running Total of Tests, Confirmed Tests & Deaths

In [11]:
chd['cumsum_number_tested'] = chd['number_tested'].cumsum()

In [12]:
chd['cumsum_number_confirmed'] = chd['number_confirmed'].cumsum()

In [13]:
chd['cumsum_number_deaths'] = chd['number_deaths'].cumsum()

In [14]:
chd['cumsum_number_hospitalized'] = chd['number_hospitalized'].cumsum()

##### Add Percentage Total of Tests Confirmed Tests Cumulative

In [15]:
chd['cumsum_pct_tested_positive'] = chd['cumsum_number_confirmed']/chd['cumsum_number_tested']

##### Add Rolling 7-Day Moving Averages of Positve Test Percentage

In [16]:
chd['7-day_rolling_avg_pct_tested_positive'] = chd['number_confirmed'].rolling(7).sum()/chd['number_tested'].rolling(7).sum()

In [77]:
chd['7-day_rolling_avg_number_tested'] = chd['number_tested'].rolling(7).sum()/7
chd['7-day_rolling_avg_number_confirmed'] = chd['number_confirmed'].rolling(7).sum()/7
chd['7-day_rolling_avg_number_hospitalized'] = chd['number_hospitalized'].rolling(7).sum()/7
chd['7-day_rolling_avg_number_deaths'] = chd['number_deaths'].rolling(7).sum()/7

In [79]:
chd.tail(10)

,extract_date,specimen_date,number_tested,number_confirmed,number_hospitalized,number_deaths,specimen_date_dt,dt,boro,pct_tested_positive,cumsum_number_tested,cumsum_number_confirmed,cumsum_number_deaths,cumsum_number_hospitalized,cumsum_pct_tested_positive,7-day_rolling_avg_pct_tested_positive,7-day_rolling_avg_number_tested,7-day_rolling_avg_number_confirmed,7-day_rolling_avg_number_hospitalized,7-day_rolling_avg_number_deaths
176,2020-08-28T00:00:00.000,2020-08-18T00:00:00.000,18852,205,15,0,2020-08-18,2020-08-28,NYC,0.010874,1911133,228199,19492,57037,0.119405,0.013266,14386.571429,190.857143,16.857143,0.857143
177,2020-08-28T00:00:00.000,2020-08-19T00:00:00.000,16919,195,18,0,2020-08-19,2020-08-28,NYC,0.011526,1928052,228394,19492,57055,0.118458,0.012620,14546.571429,183.571429,17.142857,0.571429
178,2020-08-28T00:00:00.000,2020-08-20T00:00:00.000,16465,156,14,0,2020-08-20,2020-08-28,NYC,0.009475,1944517,228550,19492,57069,0.117536,0.011977,14706.714286,176.142857,16.714286,0.428571
179,2020-08-28T00:00:00.000,2020-08-21T00:00:00.000,16235,178,17,1,2020-08-21,2020-08-28,NYC,0.010964,1960752,228728,19493,57086,0.116653,0.011571,14766.142857,170.857143,15.428571,0.428571
180,2020-08-28T00:00:00.000,2020-08-22T00:00:00.000,7194,83,10,0,2020-08-22,2020-08-28,NYC,0.011537,1967946,228811,19493,57096,0.116269,0.011401,14572.857143,166.142857,14.428571,0.142857
181,2020-08-28T00:00:00.000,2020-08-23T00:00:00.000,5924,78,3,0,2020-08-23,2020-08-28,NYC,0.013167,1973870,228889,19493,57099,0.115960,0.011222,14473.714286,162.428571,13.285714,0.142857
182,2020-08-28T00:00:00.000,2020-08-24T00:00:00.000,13747,117,5,0,2020-08-24,2020-08-28,NYC,0.008511,1987617,229006,19493,57104,0.115216,0.010615,13619.428571,144.571429,11.714286,0.142857
183,2020-08-28T00:00:00.000,2020-08-25T00:00:00.000,7408,72,14,1,2020-08-25,2020-08-28,NYC,0.009719,1995025,229078,19494,57118,0.114825,0.010478,11984.571429,125.571429,11.571429,0.285714
184,2020-08-28T00:00:00.000,2020-08-26T00:00:00.000,2728,51,0,0,2020-08-26,2020-08-28,NYC,0.018695,1997753,229129,19494,57118,0.114693,0.010545,9957.285714,105.000000,9.000000,0.285714
185,2020-08-28T00:00:00.000,2020-08-27T00:00:00.000,93,0,0,0,2020-08-27,2020-08-28,NYC,0.000000,1997846,229129,19494,57118,0.114688,0.010857,7618.428571,82.714286,7.000000,0.285714


In [18]:
boro = 'NYC'
container = "The boro chosen by user was: " + (boro)
container

'The boro chosen by user was: NYC'

In [21]:
"Total COVID-19 "+ boro + " Cases by Day"

'Total COVID-19 NYC Cases by Day'

In [22]:
hosp = {'number_hospitalized': 'Hospitalizatons'}

In [36]:
hosp['number_hospitalized']

'Hospitalizatons'

In [37]:
chd['dt'].max()

'2020-08-28'

In [38]:
chd['number_confirmed'].sum()

229129

In [44]:
total_cases = format(chd['number_confirmed'].sum(), ',')
total_deaths = chd['number_deaths'].sum()
total_hospitalizations = chd['number_hospitalized'].sum()
total_tests = chd['number_tested'].sum()

In [46]:
topline_values = ['total_cases', 'total_tests', 'total_hospitalizations', 'total_deaths']
topline_data = (total_cases, total_deaths, total_hospitalizations, total_tests)

In [64]:
df = pd.DataFrame(data=topline_data).T
#df.columns = topline_values

In [65]:
df.head()

,0,1,2,3
0,"229,129",19494,57118,1997846


In [63]:
chd['number_deaths'].sum()

19494

##### Export Data to Local Folder

In [17]:
chd.to_csv('data_output/chd.csv', index=False)

#### Build Dashboard

##### App Layout

In [33]:
app = dash.Dash(__name__)
app.layout = html.Div([
    
    html.H1("NYC COVID-19 Overall Confirmed Cases", style={"text-align":"left"}),
    
    dcc.Dropdown(id="select_boro",
                options=[
                    {"label": "NYC", "value":'NYC'},
                    {"label": "Brooklyn", "value":'Brooklyn'},
                    {"label": "Bronx", "value":'Bronx'},
                    {"label": "Manhattan", "value":'Manhattan'},
                    {"label": "Queens", "value":'Queens'},
                    {"label": "Staten Island", "value":'SI'}],
                 value= "NYC",
                 #multi-False,
                 style={"width":"40%"}
                ),
    html.Div(id="output_container", children=[]),
    html.Br(),
    dcc.Graph(id="covid_19_chart", figure={})
    
    
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/Users/cartneythompson/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


---

### COVID-19 Outcomes by Testing Cohorts: Cases, Hospitalizations, and Deaths by Boro and Cohort

#### Import Data

In [18]:
chd_boro = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/boro/boroughs-case-hosp-death.csv')

In [21]:
chd_boro.tail()

,DATE_OF_INTEREST,BK_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BX_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,MN_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,QN_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,SI_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT
174,08/21/2020,61,7,0,41,14,0,40,1,0,48,5,2,10,0,0
175,08/22/2020,19,0,1,16,4,1,17,5,1,14,3,1,4,0,1
176,08/23/2020,24,5,0,12,2,3,4,0,0,19,4,0,3,0,0
177,08/24/2020,23,0,1,16,1,1,20,0,0,14,0,0,4,0,0
178,08/25/2020,4,0,0,2,0,0,1,0,0,1,0,0,0,0,0


In [20]:
chd_boro['BK_CASE_COUNT'].sum() + chd_boro['BX_CASE_COUNT'].sum() + chd_boro['MN_CASE_COUNT'].sum() + chd_boro['QN_CASE_COUNT'].sum() + chd_boro['SI_CASE_COUNT'].sum()

228902

In [ ]:
chd_boro['BK_CASE_COUNT'].sum() + chd_boro['BX_CASE_COUNT'].sum() + chd_boro['MN_CASE_COUNT'].sum() + chd_boro['QN_CASE_COUNT'].sum() + chd_boro['SI_CASE_COUNT'].sum()